# Constructing Hamiltonain for momentum blocks for the Square Spin Ice Model

In [22]:
using LinearAlgebra
using SparseArrays
using Arpack

In [23]:
L=4

4

In [24]:
function coordinate(n;L::Int64=2)
    num_sites = L^2
    i::Int64 = Int(ceil(n/L))
    j::Int64 = mod1(n,L)  #site i is at i-th row, j-th column
    return (i,j)
end

function bit_pos(coordinate::Tuple{Int64,Int64};L::Int64=2)
    n = (coordinate[1]-1)*L + coordinate[2]
    return n
end

bit_pos (generic function with 1 method)

## Constructing neighbour list for a position in the lattice

For simplicity, we consider the transverse Ising Hamiltonian for the simplest 2D square lattice only.

In [25]:
function neib(n::Int64;L::Int64=2)
    coord = coordinate(n,L=L)
    neibs = Tuple{Int64,Int64}[]
    push!(neibs, (mod1(coord[1]+1,L), coord[2]))
    push!(neibs, (mod1(coord[1]-1,L), coord[2]))
    push!(neibs, (coord[1], mod1(coord[2]+1,L)))
    push!(neibs, (coord[1], mod1(coord[2]-1,L)))
    #=
    push!(neibs, (mod1(coord[1]+1,L), mod1(coord[2]-1,L)))
    push!(neibs, (mod1(coord[1]-1,L), mod1(coord[2]+1,L)))
    push!(neibs, (mod1(coord[1]+1,L), mod1(coord[2]+1,L)))
    push!(neibs, (mod1(coord[1]-1,L), mod1(coord[2]-1,L)))=#
    #=convert coordinations to positions in bits=#
    neibs_bit_pos = Set{Int64}()
    for neib in neibs
        push!(neibs_bit_pos, bit_pos(neib,L=L))
    end
    return neibs_bit_pos
end

neib_list = Set{Int64}[]
for n in 1:L^2
    push!(neib_list, neib(n, L=L))
end
println(neib_list)

Set{Int64}[Set([4, 13, 2, 5]), Set([14, 3, 6, 1]), Set([7, 4, 2, 15]), Set([3, 16, 8, 1]), Set([9, 8, 6, 1]), Set([7, 10, 2, 5]), Set([3, 11, 8, 6]), Set([7, 4, 5, 12]), Set([13, 10, 5, 12]), Set([9, 14, 11, 6]), Set([7, 10, 15, 12]), Set([9, 16, 11, 8]), Set([9, 14, 16, 1]), Set([13, 10, 2, 15]), Set([14, 3, 16, 11]), Set([4, 13, 15, 12])]


### In order to construct a list of reference states, we first need to define the symmetry of the system, as well as the symmetry operation on the system

In [26]:
function translation(state::Int64; direction::Symbol, dist::Int64, L::Int64)
    max_len = L^2
    state_binary = digits!(zeros(Int64, 64), state, base = 2)
    translated_state = 0
    if direction == :x
        for pos in 1:max_len
            if state_binary[pos] == 1
                #translated_pos = mod1(pos + dist, L) + Int(floor((pos-0.5)/L))*L
                #translated_state = translated_state ⊻ (1<<(translated_pos-1))
                pos_coord = coordinate(pos,L=L)
                trans_pos_coord_i = pos_coord[1]
                trans_pos_coord_j = mod1(pos_coord[2] + dist, L)
                trans_pos = bit_pos((trans_pos_coord_i, trans_pos_coord_j), L=L)
                translated_state = translated_state ⊻ (1<<(trans_pos-1))
            end
        end
        return translated_state
    elseif direction == :y
        for pos in 1:max_len
            if state_binary[pos] == 1
                #translated_pos = Int((mod1(ceil((pos-0.5)/L) + dist, L)-1)*L + mod1(pos, L))
                #translated_state = translated_state ⊻ (1<<(translated_pos-1))
                pos_coord = coordinate(pos,L=L)
                trans_pos_coord_i = mod1(pos_coord[1] + dist, L)
                trans_pos_coord_j = pos_coord[2]
                trans_pos = bit_pos((trans_pos_coord_i, trans_pos_coord_j), L=L)
                translated_state = translated_state ⊻ (1<<(trans_pos-1))
            end
        end
        return translated_state
    else
        error("Direction not defined.")
    end
end

translation (generic function with 1 method)

### check routine to check whether a state s belong to a certain k subspace, as well as determining the periodicity of the state under translation of 2 different directions.

In [27]:
function checkstate(s::Int64; k::Real, direction::Symbol, L::Int64)
    R = -1; t = s;
    for i in 1:L
        t = translation(t, direction=direction, dist=1; L=L)
        if t < s
            #println("t<s")
            return (false, R)
        elseif t == s
            R = i
            if mod(k, L/i) != 0
                #println("$(L/i)")
                return (false, R)
            else
                #println("should be true")
                return (true, R)
            end
        end
    end
end

#Check whether a state is ref state 2D, if it's a ref state, return number of different states Da
function checkstate(s::Int64; k::Tuple, L::Int64, ϵ::Float64 = 10^(-10))
    k = 2pi/L .* k
    states = Set{Int64}() 
    push!(states, s)
    is_ref = true
    Fa = 0
    t = s
    for i in 1:L
        t = translation(t, direction=:x, dist=1; L=L)
        for j in 1:L
            t = translation(t, direction=:y, dist=1; L=L)
            push!(states, t)
            # Check if the state is the smallest integer and return periodicity Rx, Ry
            if t < s
                is_ref = false                
            end
            if t == s
                Fa += exp(-im*(k[1]*i + k[2]*j))
            end
            #finished check states for one certain translation
        end
    end
    if abs(Fa) < ϵ
       is_ref = false
    end
    return (is_ref, length(states), real(Fa))
end


checkstate (generic function with 1 method)

### Now we can construct a few list of reference states for different ks. Along with its periodicity. 

We represent a k value by an integer $n = {1, 2, ..., L_T}$ multiplying by $\frac{2\pi}{R}$ with $R$ being the periodicity of the state. For $R\ne L_T$, allowed k values are $\frac{2\pi}{N} \frac{N}{R} n$.So we have to choose integer from the list that is a multiple of $\frac{N}{R}$.

For a lattice that is more complicated, $L_T$ should equal to the period of the lattice on x or y direction, not neccesarily equal to the number of sites on each direction $L$. For example the square ice ising model lattice would have L equal to half the number of sites each direction, because the lattice repeats itself only twice the sites.

In [28]:
# generate a list of ref states for a 2D system, with k being a 2D Tuple (kx, ky) 
# return (reference state, number of different states for this reference state, sum of phase for the k state)
function ref_state_gen(;k::Tuple, L::Int64)
    ref_states = Int64[]#Set{Int64}()
    Da = Int64[]
    Fa = Number[]
    ref_state_nums = Dict{Int64, Int64}()
    ref_state_tots = 0
    for state in 0:(2^(L^2)-1)
        chk_state = checkstate(state, k=k, L=L)
        if chk_state[1]
            ref_state_tots += 1
            ref_state_nums[state] = ref_state_tots
            push!(ref_states, state)
            push!(Da, chk_state[2])
            push!(Fa, chk_state[3])
        end
    end
    return Dict(:ref_states => ref_states, :Da => Da, :Fa => Fa, :ref_state_tots => ref_state_tots, :ref_state_nums => ref_state_nums)
end



ref_state_gen (generic function with 1 method)

### For each state, we also need to build a function to find its corresponding reference state and how it can be translated to a momentum state

In [29]:
# return the number of translation in x and y direction in a Tuple (i, j)
function find_ref(s::Int64; L::Int64)
    t = s
    ref = t
    trans = (0,0)
    for i in 1:L
        t = translation(t, direction=:x, dist=1; L=L)
        for j in 1:L
            t = translation(t, direction=:y, dist=1; L=L)
            if t < ref
                ref = t
                trans = (i, j)  
            end
        end
    end
    return Dict(:trans => trans, :ref => ref)
end
find_ref(9; L=2)

Dict{Symbol,Any} with 2 entries:
  :trans => (1, 2)
  :ref   => 6

### Now construct Hamiltonian for each momentum sector using the previously generated momentum states

The Hamiltonian is:
$H=J\sum_{i=1}^{N}(S_i^z S_{i+1}^z + 1/2(S_i^+ S_{i+1}^- + S_i^- S_{i+1}^+)) $


We know that the K-space hamiltonian for the j-th bond among all the bonds is :

$<b_{i\sigma}(\vec{k})|H_{i\sigma}|a(\vec{k})> = h_{i\sigma}(a) e^{-\vec{l_{i\sigma}} \cdot \vec{k}} \sqrt{\frac{N_{b_{i\sigma}}}{N_a}}$

So the Hamiltonian corresponding to the $|a(\vec{k})>$ column should be the sum of all the above terms over $i,\sigma$.

In [30]:
function Hamiltonian(;k::Tuple{Int64, Int64},L::Int64,J=1, h=1, neib_list, state_list, state_nums, state_tot, Das, Fas)
    H = zeros(Complex{Float64}, state_tot,state_tot)
    for state in state_list #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        Na = Das[state_nums[state]]*abs(Fas[state_nums[state]])^2 # for calculation of Hamiltonian later
        for i in 1:L^2 #loop over all sites i in a given state
            #all terms are spin pairs, no h field, so no single site term
            for j in neib_list[i] #loop over(compare) all neighbors of a given site
                if state_binary[i]==state_binary[j]
                    H[state_nums[state],state_nums[state]] += J/4/2
                else
                    H[state_nums[state],state_nums[state]] -= J/4/2
                    flipped_state = state ⊻ (1<<(i-1))
                    flipped_state = flipped_state ⊻ (1<<(j-1))
                    ref_flipped_state = find_ref(flipped_state, L = L)
                    if haskey(state_nums, ref_flipped_state[:ref]) # check if the flipped state is within this k sector
                        # calculated the contribution to the Hamiltonian
                        Nb = Das[state_nums[ref_flipped_state[:ref]]]*abs(Fas[state_nums[ref_flipped_state[:ref]]])^2
                        H[state_nums[state],state_nums[ref_flipped_state[:ref]]] += 1/2 * J/2 * exp(2pi*im*(k[1]*ref_flipped_state[:trans][1] + k[2]*ref_flipped_state[:trans][2]) / L) * sqrt(Nb / Na)
                    end
                end   
            end
        end
    end
    return H    
end

function hermitian_test()
    eigs = []
    for kx in 1:L
        for ky in 1:L
            ref_state = ref_state_gen(k=(kx,ky),L=L)
            H = Hamiltonian(;k=(kx,ky),L=L,J=1, h=0, neib_list=neib_list, state_list=ref_state[:ref_states], state_nums=ref_state[:ref_state_nums], state_tot=ref_state[:ref_state_tots], Das=ref_state[:Da], Fas=ref_state[:Fa])
            if maximum(abs.(Hermitian(H, :U) - Hermitian(H, :L))) > 10^(-10)
                return false
            end
            push!(eigs, eigen(Hermitian(H,:U)).values...)
        end
    end
    return eigs
end

eigens = hermitian_test()
#=
ref_state = ref_state_gen(k=(4,4),L=L)
H = Hamiltonian(;k=(4,4),L=L,J=1, h=0, neib_list=neib_list, state_list=ref_state[:ref_states], state_nums=ref_state[:ref_state_nums], state_tot=ref_state[:ref_state_tots], Das=ref_state[:Da], Fas=ref_state[:Fa])
println(eigen(Hermitian(H,:U)).values)=#

65536-element Array{Any,1}:
 -8.518283596238055 
 -8.518283596238055 
 -8.518283596238055 
 -7.812148008952249 
 -7.477053302195181 
 -7.477053302195181 
 -7.477053302195181 
 -7.477053302195181 
 -7.477053302195181 
 -7.432605406057665 
 -7.432605406057665 
 -7.432605406057663 
 -7.1285233746080845
  ⋮                 
  7.999999999999998 
  7.999999999999998 
  7.999999999999998 
  7.999999999999998 
  7.999999999999998 
  8.0               
  8.0               
  8.0               
  8.0               
  8.000000000000002 
  8.000000000000002 
  8.000000000000002 

In [31]:
using LinearAlgebra
minimum(Array{Float64,1}(eigens))

-11.228483208428859

In [10]:
kx=0;ky=0
ref_state = ref_state_gen(k=(kx,ky),L=L)
H = Hamiltonian(;k=(kx,ky),L=L,J=1, h=0, neib_list=neib_list, state_list=ref_state[:ref_states], state_nums=ref_state[:ref_state_nums], state_tot=ref_state[:ref_state_tots], Das=ref_state[:Da], Fas=ref_state[:Fa])
eigen(Hermitian(H))

Eigen{Complex{Float64},Float64,Array{Complex{Float64},2},Array{Float64,1}}
eigenvalues:
4156-element Array{Float64,1}:
 -8.678844429796028 
 -7.051601126984172 
 -7.051601126984172 
 -6.568884154599273 
 -5.83784499831572  
 -5.723466734266176 
 -5.723466734266174 
 -5.663737356210973 
 -5.66373735621097  
 -5.211385774381115 
 -5.1226264783020685
 -5.044131207969581 
 -4.984096116025211 
  ⋮                 
  4.999999999999998 
  4.999999999999999 
  5.451605962955777 
  5.45160596295578  
  5.484758976203636 
  5.484758976203639 
  6.166207393881133 
  6.166207393881135 
  7.0               
  7.0               
  8.0               
  8.0               
eigenvectors:
4156×4156 Array{Complex{Float64},2}:
          0.0+0.0im           0.0+0.0im  …  0.0+0.0im  1.0+0.0im  0.0+0.0im
          0.0+0.0im           0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
  9.06784e-51+0.0im  -1.53765e-51+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 -1.37506e-51+0.0im  -1.15565e-50+0.0im     0.0+0.0im  0

3-element Array{Any,1}:
 2
 3
 4